# Running Chronos on AutoGluon TimeSeries

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/master/docs/tutorials/timeseries/forecasting-chronos.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/master/docs/tutorials/timeseries/forecasting-chronos.ipynb)

AutoGluon-TimeSeries (AG-TS) now features [Chronos](https://github.com/amazon-science/chronos-forecasting), a family of pretrained time series forecasting models. Chronos models are based on language model architectures, and work by quantizing time series into buckets which are treated as tokens. Language models are then trained on these token sequences using cross-entropy loss. 
The current iteration of Chronos models, [available](https://huggingface.co/amazon/chronos-t5-large) on Hugging Face 🤗, are based on the T5 architecture and were trained on a large corpus of open-source time series data augmented with synthetic data generation techniques. The Chronos [paper](https://arxiv.org/abs/2403.07815) provides greater detail about the models and how they were trained. 

AG-TS provides a robust and easy way to use Chronos as well as combine it with other powerful time series models to build highly accurate ensembles for forecasting. 
Chronos is available through the familiar AG-TS API, and is part of `"best_quality"` and `"high_quality"` presets of `TimeSeriesPredictor`. 
Moreover, the `TimeSeriesPredictor` now supports custom presets for using only the Chronos model for forecasting. 

In [ ]:
!pip install autogluon.timeseries
!pip uninstall torchaudio torchvision torchtext -y  # fix incompatible package versions on Colab

In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

# Getting Started with Chronos

Chronos is available in 5 model sizes. These are `tiny`, `mini`, `small`, `base` and `large`, and are encoder-decoder transformer models with 8M, 20M, 46M, 200M and 710M parameters respectively. Being pretrained models for zero-shot forecasting, Chronos is different from other models available in AG-TS. 
Specifically, Chronos models do not really `fit` time series data. However, when `predict` is called, they carry out relatively more expensive computation which scales linearly with the number of time series that should be forecasted. In this aspect, they behave like local statistical models such as ETS or ARIMA, where expensive computation happens during inference. Differently from statistical models, however, computation in the larger Chronos models requires an accelerator chip to run in a reasonable amount of time.

The easiest way to get started with Chronos on the `TimeSeriesPredictor` is through model-specific presets. As of v1.1, the `TimeSeriesPredictor.fit` method has Chronos presets specific to the model size, such as `chronos_small`, or `chronos_base`. 
There are also two more presets that allow ensembling Chronos with other AG-TS models: `chronos_ensemble` and `chronos_large_ensemble`, which include Chronos-small and Chronos-large respectively. Finally, the `high_quality` and `best_quality` presets include Chronos-base.

Note that the `TimeSeriesPredictor` will refuse to run model sizes larger than `mini` without a GPU. However, it can work with `tiny` and `mini` on the CPU as well. 

Let's work with a subset of the M4 competition data set to see Chronos-tiny in action.

In [ ]:
df = TimeSeriesDataFrame(
    "https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly_subset/train.csv"
)
df.head()

In [ ]:
training_window = df.slice_by_timestep(start_index=None, end_index=-5)
predictor = TimeSeriesPredictor(prediction_length=5).fit(
    training_window, presets="chronos_tiny",
)

As promised, Chronos does not take any time to `fit`. The `fit` call merely serves as a proxy for the `TimeSeriesPredictor` to do some of its chores under the hood, such as inferring the frequency of time series, caching data and saving the predictor's state. 

Let's use the predictor to generate forecasts, and the `TimeSeriesPredictor.plot` method to visualize them.

In [ ]:
predictions = predictor.predict(training_window)
predictor.plot(
    data=df.slice_by_timestep(-200, None), 
    predictions=predictions, 
    item_ids=["H1", "H2"],
)

## Configuring for Performance

Looks good! As with all large deep learning models, however, some fine-grained control of inference parameters can be needed to both optimize performance and avoid out-of-memory issues on specific hardware. For this, we will need to dive a bit deeper, configuring `hyperparameters` of the `TimeSeriesPredictor` directly.

In [ ]:
%%time
predictions = TimeSeriesPredictor(prediction_length=5).fit(
    training_window,
    hyperparameters={
        "Chronos": {
            "model_path": "tiny",
            "batch_size": 64,
            "torch_dtype": "bfloat16",
            "device": "cpu",
        }
    },
    skip_model_selection=True,
    verbosity=0,
).predict(
    training_window,
)

Above, we explicitly told `TimeSeriesPredictor` to skip running backtests during `fit` via `skip_model_selection=True`, as we will only consider a single model.
In `hyperparameters`, `batch_size` configures the number of time series used per batch during inference. 
`torch_dtype="bfloat16"` instructs Chronos to use half-precision floating point for reduced memory footprint.
On the GPU, and for larger model sizes, these parameters can greatly improve inference performance for Chronos based on the hardware used.

On the CPU, the adjustments above appear to be helping only very slightly. The computation took less than a third of a second per time series. However, the AutoGluon implementation of Chronos also supports using deep learning compilers which improve performance on Intel CPUs. Providing the `optimization_strategy="openvino"` hyperparameter instructs AutoGluon to first convert Chronos for use with the [OpenVINO](https://github.com/openvinotoolkit/openvino) runtime. Note that this requires installing AG-TS with OpenVINO and is not included by default.

Below, we use two tricks. We first `fit` and `persist` the model. The `TimeSeriesPredictor.persist` method tells AutoGluon to prepare Chronos for OpenVINO and keep it in device memory for fast, on-demand inference. `optimization_strategy="openvino"` tells Chronos to work with OpenVINO. Behind the scenes, AutoGluon will use Hugging Face [optimum](https://github.com/huggingface/optimum-intel) for this conversion. 

In [ ]:
%%capture
!pip install "optimum[openvino]"

In [ ]:
%%capture
predictor = TimeSeriesPredictor(prediction_length=5).fit(
    training_window,
    hyperparameters={
        "Chronos": {
            "model_path": "tiny",
            "batch_size": 64,
            "device": "cpu",
            "optimization_strategy": "openvino",
        }
    },
    verbosity=0,
    skip_model_selection=True,
)
predictor.persist()

In [ ]:
%%time
predictions = predictor.predict(training_window)

That seems to have cut the inference time by more than half!

We could have also used the ONNX runtime by providing `optimization_strategy="onnx"`. For a discussion of these and other hyperparameters of Chronos, see the Chronos model [documentation](forecasting-model-zoo.md).

## FAQ


#### How accurate is Chronos?

In several independent experiments we found Chronos to be effective in zero-shot forecasting. 
Chronos-large's accuracy appears to generally outperform statistical baseline models in accuracy, and comparable with deep learning 
models such as `TemporalFusionTransformer` or `PatchTST`.

#### What hardware do larger Chronos models require?

We tested Chronos on AWS `g4dn.2xlarge` and `p3.2xlarge` instances, which feature NVIDIA T4 Tensor Core and NVIDIA Tesla 
V100 GPUs respectively, with at least 16GiB of GPU memory and 32GiB of main memory.

#### Can I fine-tune Chronos?

The current iteration of Chronos on AutoGluon does not support fine tuning, although we will provide this functionality in 
later versions of AutoGluon.

#### Does Chronos work with covariates or features?

The current iteration of Chronos does not support covariate or feature use, however we will provide this functionality in 
later versions. In the meanwhile, presets such as `chronos_ensemble` feature models which do take advantage of features.

#### Where can I ask specific questions on Chronos?

The AutoGluon team are among the core developers of Chronos. So you can ask Chronos-related questions on AutoGluon channels such 
as the Discord [server](https://discord.gg/wjUmjqAc2N), or [GitHub](https://github.com/autogluon/autogluon). You can also join 
the discussion on the Chronos GitHub [page](https://github.com/amazon-science/chronos-forecasting/discussions).